In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install bitsandbytes # 모델을 경량화해서 로드
!pip install transformers
!pip install peft # LoRA 튜닝을 하게 해주는 라이브러리
!pip install accelerate
!pip install trl
!pip install datasets

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from trl import SFTTrainer
import os
import re
import warnings
warnings.filterwarnings("ignore")

### preprocess

In [4]:
# train_df = pd.read_csv('../train.csv')
train = pd.read_csv('/content/drive/MyDrive/데이콘/KoSOLAR/train_preprocess_data_aug.csv')
train

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
...,...,...
6997,현관문 주위 결로의 대책은 뭐야? 또한 현관문 주위 결로의 원인은 뭐야??,현관문 주위 결로의 대책은 규격제품을 사용하고 방화문 현관문 내부 종이 허니컴 사이...
6998,"부엌 및 욕실의 결로의 원인은 뭐야? 또한 AD, PD에 면한 벽체 결로에 대한 대...",부엌이나 욕실의 경우 외벽이나 비 난방공간에 접한 면이 벽체의 일부인 부엌이나 욕실...
6999,발코니에 결로가 발생하는 원인은 뭐야? 또한 벽장 부위 결로의 원인이 뭐야??,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...
7000,부엌 및 욕실의 결로의 대책은 뭐야? 또한 외벽 모서리 부위에 결로가 발생하는 원인...,부엌 및 욕실의 결로에 대한 대책은 환기구 설치입니다. 부엌 및 욕실의 결로는 주로...


### train

In [5]:
# output_dir = "KoSOLAR_finetuned_preprocess"
output_dir = "/content/drive/MyDrive/데이콘/KoSOLAR/KoSOLAR_finetuned_preprocess"
model_id = 'kimwooglae/WebSquareAI-Instruct-KoSOLAR-10.7b-v0.5.34'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 모델을 쪼개서 load함

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(48000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [7]:
tokenizer

LlamaTokenizerFast(name_or_path='kimwooglae/WebSquareAI-Instruct-KoSOLAR-10.7b-v0.5.34', vocab_size=48000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '<|im_end|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
}

In [ ]:
# "아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{Question}\n\n### 답변:\n{answer}<|im_end|>"

In [8]:
dataset = []
for question, answer in tqdm(zip(train['질문'], train['답변']), total = len(train)):
    dataset.append(f"아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{question}\n\n### 답변:\n{answer}"+tokenizer.eos_token)
dataset = Dataset.from_dict({"text": dataset})
dataset = dataset.shuffle()
dataset

  0%|          | 0/7002 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 7002
})

In [9]:
dataset['text'][0]

'아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n포세린 타일이 뭐야?\n\n### 답변:\n포세린 타일이란 점토를 주원료로 만들어진 무광의 타일을 의미합니다. 가격은 평당 10~20만원입니다. 여름에는 쉬원하고 겨울에는 따뜻하며 내구성이 좋고 바닥 울림이 적다는 장점이 있지만 가격이 조금 비싼 편이며 충격 흡수율이 낮아 노약자가 있는 집안에는 적절하지 않다는 단점이 있습니다.<|im_end|>'

In [10]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
lora_config_dict = {
    'lora_r' : 8,
    'lora_alpha' : 32,
    'lora_dropout' : 0.05,
    'lora_bias' : "none",
    'lora_task_type' : "CAUSAL_LM",
    'target_modules': ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"]
}

peft_config = LoraConfig(
            lora_alpha=lora_config_dict['lora_alpha'],
            lora_dropout=lora_config_dict['lora_dropout'],
            r=lora_config_dict['lora_r'],
            bias=lora_config_dict['lora_bias'],
            task_type=lora_config_dict['lora_task_type'],
            target_modules = lora_config_dict['target_modules']
        )

In [12]:
train_param = {
    'num_train_epochs': 5,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "adamw_torch",
    'save_steps': 1000,
    'logging_steps': 10,
    'learning_rate': 0.0002,
    'weight_decay':  0.01,
    'max_grad_norm': 1,
    'warmup_ratio': 0.06,
    'group_by_length' : False,
    'lr_scheduler_type': 'cosine'
}

training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=train_param['num_train_epochs'],
            per_device_train_batch_size=train_param['per_device_train_batch_size'],
            per_device_eval_batch_size=train_param['per_device_eval_batch_size'],
            gradient_accumulation_steps=train_param['gradient_accumulation_steps'],
            optim=train_param['optim'],
            save_steps=train_param['save_steps'],
            logging_steps=train_param['logging_steps'],
            learning_rate=train_param['learning_rate'],
            weight_decay=train_param['weight_decay'],
            # fp16=kwargs['fp16'],
            # bf16=kwargs['bf16'],
            max_grad_norm=train_param['max_grad_norm'],
            # max_steps=kwargs['max_steps'],
            warmup_ratio=train_param['warmup_ratio'],
            group_by_length=train_param['group_by_length'],
            lr_scheduler_type=train_param['lr_scheduler_type'],
#             report_to="tensorboard"
        )

In [13]:
trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_config,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            # neftune_noise_alpha=5,
            # callbacks=[AbortCallback()]
            )

Map:   0%|          | 0/7002 [00:00<?, ? examples/s]

In [14]:
model.config.use_cache = False
# trainer.train(resume_from_checkpoint=False)
trainer.train(resume_from_checkpoint=True)

Step,Training Loss
5010,0.055200
5020,0.052000
5030,0.065300
5040,0.071600
5050,0.074200
5060,0.079000
5070,0.081700
5080,0.077100
5090,0.076100
5100,0.077600


TrainOutput(global_step=8755, training_loss=0.02853915849322935, metrics={'train_runtime': 24368.1292, 'train_samples_per_second': 1.437, 'train_steps_per_second': 0.359, 'total_flos': 8.482998017356923e+17, 'train_loss': 0.02853915849322935, 'epoch': 5.0})

In [15]:
trainer.model.save_pretrained(output_dir)